In [31]:
%pip install -q kaggle
%pip install -q kagglehub
%pip install -q dask
%pip install -q ipywidgets --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [32]:
import pandas as pd, requests
import kagglehub, re, json, os
import requests, subprocess
import sys, time
from tqdm import tqdm

In [33]:
# Définis le chemin du fichier kaggle.json
path = r"C:\Credentials\kaggle.json"
#json file : {"username":"flosrv","api_key":"44c79a4d5d1534092d24ee29f34e6ee3"}
# Charge les credentials depuis le fichier JSON
with open(path, 'r') as file:
    content = json.load(file)

# Récupère les valeurs du fichier
username = content["username"]
api_key = content["key"]  # ⚠️ Assure-toi que la clé s'appelle bien "key" dans ton fichier JSON

# ✅ Définis les variables d'environnement avant de les utiliser
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = api_key
import kaggle
from kaggle import KaggleApi
from kaggle import api # import the already authenticated API client
# 1. **Authentification avec Kaggle**
def authenticate_kaggle():
    # Vérifie si le fichier kaggle.json est dans le bon dossier
    kaggle_json_path = os.path.expanduser(path)  # pour Linux/macOS
    # Pour Windows, utilisez : os.path.expanduser(r'C:\Users\<Votre-Nom>\.kaggle\kaggle.json')

    # Si le fichier n'existe pas, avertir l'utilisateur
    if not os.path.exists(kaggle_json_path):
        print("Le fichier 'kaggle.json' n'est pas trouvé. Téléchargez-le depuis votre compte Kaggle.")
        print("Assurez-vous de le placer dans le répertoire ~/.kaggle/ (Linux/macOS) ou C:\\Users\\<Your-Username>\\.kaggle\\ (Windows).")
        return False  # L'authentification échoue
    else:
        print(f"Authentification réussie avec le fichier '{kaggle_json_path}'")
        api = KaggleApi()
        api.authenticate()  # Authentification avec le fichier 'kaggle.json'
        return True

try:
    # Exécuter l'authentification
    authenticate_kaggle()
except Exception as e:
    print(f"Erreur lors de l'authentification : {e}")
    sys.exit(1)

Authentification réussie avec le fichier 'C:\Credentials\kaggle.json'


In [34]:
path = r"C:\Users\f.gionnane\Downloads\Mes-Projets\Kaggle Data Projects\data"

Functions For Datasets

In [ ]:
def list_datasets(keyword=None, usability_min=None, min_mb=None, columns=None):
    print(f"Lister les datasets correspondant à '{keyword if keyword else 'tous les datasets'}' :")
    
    """
    dataset metadata:
    'id', 'ref', 'subtitle', 'creatorName', 'creatorUrl', 'totalBytes',
        'url', 'lastUpdated', 'downloadCount', 'licenseName', 'description',
        'ownerName', 'ownerRef', 'kernelCount', 'title', 'viewCount',
        'voteCount', 'currentVersionNumber', 'usabilityRating', 'tags'
    """
    
    # Traitement des mots-clés
    if keyword:
        if isinstance(keyword, str):
            keyword = [keyword]
        keyword = [re.escape(kw) for kw in keyword]

    # Recherche des datasets via l'API Kaggle
    datasets = []
    for kw in keyword:
        datasets.extend(api.dataset_list(search=kw))

    result = []
    for dataset in datasets:
        dataset_dict = dataset.to_dict()  # Convertir le dataset en dictionnaire
        
        title = dataset_dict.get("title", "")
        ref = dataset_dict.get("ref", "")
        description = dataset_dict.get("description", "")
        lastUpdated = dataset_dict.get("lastUpdated", "")
        voteCount = dataset_dict.get("voteCount", 0)
        tags = dataset_dict.get("tags", [])
        usabilityRating = dataset_dict.get("usabilityRating", 0)

        # Convertir la taille en Mo
        dataset_size = dataset_dict.get("totalBytes", 0)
        dataset_size_mb = dataset_size / (1024 * 1024)

        # Filtrage selon la taille du dataset
        if min_mb is not None and dataset_size_mb < min_mb:
            continue

        # Filtrage selon la note d'utilisabilité
        if usability_min is not None and usabilityRating < usability_min:
            continue

        # Si des colonnes spécifiques sont demandées
        if columns is not None:
            dataset_dict = {key: dataset_dict[key] for key in columns if key in dataset_dict}

        # Ajouter les informations dans le résultat
        result_dict = {
            "title": title,
            "ref": ref,
            "description": description,
            "lastUpdated": lastUpdated,
            "voteCount": voteCount,
            "tags": tags,
            "usabilityRating": usabilityRating,
            "dataset_size_mb": round(dataset_size_mb,2)
        }

        result.append(result_dict)

    # Création du DataFrame
    df = pd.DataFrame(result)
    
    print(f"\n{df.shape[0]} datasets trouvés.")
    return df

keywords = [
    "sales",          # Ventes : analyses de performance commerciale
    "marketing",      # Campagnes, ROI, funnel, etc.
    "customer",       # Données clients : segmentation, churn, satisfaction
    "finance",        # Budgets, prévisions, états financiers
    "supply chain",   # Logistique, stocks, prévision de demande
    "human resources",# RH : turnover, performance, recrutement
    "retail",         # Analyse de produits, points de vente, inventaire
    "ecommerce",      # Données web, comportement d'achat, panier moyen
    "operations",     # Optimisation de processus internes
    "real estate"     # Immobilier : investissements, prévisions de prix
]

def download_dataset(dataset_ref: str, path: str = "."):

    """
    Télécharge un dataset Kaggle via sa ref en créant un dossier pour chaque dataset, avec une barre de progression.

    Args:
        dataset_ref (str): Référence du dataset sous la forme "username/dataset-name".
        path (str, optionnel): Répertoire où enregistrer le fichier téléchargé. Par défaut: ".".

    Returns:
        None
    """


    dataset_folder = os.path.join(".", dataset_ref.replace("/", "_"))
    if os.path.exists(dataset_folder):
        print(f"�� Le dataset '{dataset_ref}' est déjà téléchargé dans le répertoire courant.")
        return
    os.makedirs(dataset_folder)  # Créer le dossier si il n'existe pas déjà
    print(f"�� Création du dossier '{dataset_folder}' pour le dataset '{dataset_ref}'...")     

    # Récupérer la taille du dataset via l'API Kaggle
    metadata_url = f"https://www.kaggle.com/api/v1/datasets/metadata/{dataset_ref}"
    try:
        response = requests.get(metadata_url)
        if response.status_code == 200:
            metadata = response.json()
            dataset_size = metadata["totalBytes"] / 1_000_000  # Conversion en Mo
            print(f"📥 Téléchargement de '{dataset_ref}' ({dataset_size:.2f} MB)...")
        else:
            print("⚠️ Impossible de récupérer la taille du dataset.")
            dataset_size = None
    except requests.exceptions.RequestException:
        print("⚠️ Erreur lors de la récupération des métadonnées.")
        dataset_size = None

    # Commande de téléchargement
    command = ["kaggle", "datasets", "download", "-d", dataset_ref, "-p", dataset_folder, "--unzip"]

    try:
        # Démarrer le téléchargement avec suivi de progression
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Barre de progression
        with tqdm(total=dataset_size, unit="MB", unit_scale=True, desc="📥 Progression") as pbar:
            while process.poll() is None:
                time.sleep(1)
                pbar.update(1)

        # Vérifier si le téléchargement a réussi
        if process.returncode == 0:
            print(f"✅ Dataset '{dataset_ref}' téléchargé avec succès dans '{dataset_folder}'.")
        else:
            print(f"❌ Erreur lors du téléchargement. Code de retour: {process.returncode}")

    except subprocess.CalledProcessError as e:
        print(f"❌ Erreur dans la commande Kaggle. Code de retour: {e.returncode}")
        print(f"Erreur : {e.stderr.decode('utf-8')}")
    except FileNotFoundError:
        print("❌ Erreur : La commande 'kaggle' n'a pas été trouvée. Installez l'API Kaggle.")
    except Exception as e:
        print(f"❌ Une erreur inattendue s'est produite : {str(e)}")

df_consulting_datasets =list_datasets(keyword=keywords, usability_min=0.6, min_mb=10)

Lister les datasets correspondant à '['sales', 'marketing', 'customer', 'finance', 'supply chain', 'human resources', 'retail', 'ecommerce', 'operations', 'real estate']' :

39 datasets trouvés.


In [45]:
df_consulting_datasets.head()

,title,ref,description,lastUpdated,voteCount,tags,usabilityRating,dataset_size_mb
0,Vehicle Sales Data,syedanwarafridi/vehicle-sales-data,,2024-02-21T20:16:17.790Z,584,"[{'ref': 'transportation', 'name': 'transporta...",1.000000,18.84
1,Telecom customer,abhinav89/telecom-customer,,2017-08-27T03:01:50.747Z,207,"[{'ref': 'business', 'name': 'business', 'desc...",0.705882,13.57
2,Customer Support on Twitter,thoughtvector/customer-support-on-twitter,,2017-12-03T23:44:27.157Z,495,"[{'ref': 'business', 'name': 'business', 'desc...",0.911765,168.58
3,US Funds dataset from Yahoo Finance,stefanoleone992/mutual-funds-and-etfs,,2021-12-11T16:06:22.453Z,253,"[{'ref': 'united states', 'name': 'united stat...",1.000000,353.34
4,Massive Yahoo Finance Dataset,iveeaten3223times/massive-yahoo-finance-dataset,,2023-11-29T17:24:31.693Z,51,"[{'ref': 'research', 'name': 'research', 'desc...",1.000000,22.78


In [ ]:
df_consulting_datasets["tags"] = df_consulting_datasets["tags"].apply(lambda tag_list: [tag.get("name", "") for tag in tag_list if not isinstance(tag_list, list)])
df_consulting_datasets.head()

In [41]:
columns= ['id',  'title', 'ref', 'totalBytes', 'usabilityRating', 
          'description', 'url', 'lastUpdated', 'topicCount', 
          'voteCount', 'currentVersionNumber', 'tags']

In [ ]:
url_list = []
metadata_list = []
count = 0

download_path = "/downloaded_datasets"

for idx, row in df_consulting_datasets.iterrows():
    url = row["url"]
    url_list.append(url)
    ref = row["ref"]
    try:
        download_dataset(ref,path=download_path)
        print(f"Dataset '{ref}' téléchargé avec succès.")
        count += 1
    except Exception as e:
        print(f"Error downloading dataset for {ref} (Error: {str(e)})")
        continue

    if response.status_code == 200:
        content = response.json()  # Stocker le JSON
        metadata_list.append(content)  # Ajouter à la liste des métadonnées
        
        # Vérifier si le dataset est privé
        if content["info"]["isPrivate"]:
            df_consulting_datasets.drop(labels=idx, inplace=True)  # Supprimer la ligne
            count += 1
            
    else:
        print(f"Error fetching metadata for {url} (Status code: {response.status_code})")

df_metadata = pd.DataFrame(metadata_list)
print(f"Nombre de datasets privés supprimés : {count}")
df_metadata.head()

KeyError: 'url'

In [ ]:
# Normaliser la colonne "info" avant d'appliquer le filtrage des colonnes
df_normalized = df_metadata.join(pd.json_normalize(df_metadata["info"]))
df_normalized.drop(columns=["info"], inplace=True)

# Appliquer le filtrage des colonnes après la normalisation
df_normalized = df_normalized.loc[:, ~df_normalized.columns.str.contains('has|nullable', case=False)]

df_normalized.head()

In [ ]:
print(df_normalized.shape)
df_normalized["isPrivate"].value_counts()

Functions For Models

In [ ]:
def list_models(keyword):
    print(f"Lister les modèles correspondant à '{keyword}' :")
    
    api = KaggleApi()
    api.authenticate()
    
    models = api.models_list(search=keyword)
    desired_columns = [
        'id', 'ref', 'title', 'description', 'url',
        'totalVotes', 'lastUpdated', 'tags', 'licenseName'
    ]
    
    result = []
    for model in models:
        model_dict = {key: model[key] for key in model if key in desired_columns}
        result.append(model_dict)
    
    df = pd.DataFrame(result)

    # Suppression des colonnes inutiles contenant "has" ou "Nullable"
    df = df.loc[:, ~df.columns.str.contains('has|Nullable', case=False)]
    df["tags"] = df["tags"].apply(lambda x: [d["name"] for d in x] if x is not None else [])
    # Vérification des liens URL pour voir si les modèles sont accessibles
    for idx, row in df.iterrows():
        url = row["url"]
        try:
            response = requests.get(url)
            if response.status_code >= 400:
                df.drop(idx, inplace=True)
        except requests.exceptions.RequestException:
            df.drop(idx, inplace=True)
    

    return df

In [ ]:
url_list =[]

for idx, row in df_ocean_datasets.iterrows():
    url = row["url"]
    url_list.append(url)
    title = row["title"]
    if title == 'OCEAN DATA / CLIMATE CHANGE / NASA':
        try:
            download_dataset(url)
        except Exception as e:
            print(e)
    else :
        print('dataset not found')